# 🧠 智能体实践教学：构建你的第一个智能体

## 📘 教学目标
本次Notebook将引导你：
1. 了解什么是智能体（Agent）及其基本结构；
2. 掌握使用 `langchain` 和 `openai` 构建一个具备基础能力的智能体；
3. 亲手实现一个能对话、调用工具并具备记忆功能的智能体。

## 🧩 Step 1：什么是智能体？

我们先来了解一下基础概念。

In [ ]:
from IPython.display import Markdown

Markdown('''
**智能体（Agent）是一个能够感知环境、做出判断并执行行动的实体。**
在大模型系统中，它通常具备：
- **语言模型（LLM）**：用于理解指令和生成回应；
- **工具（Tools）**：用于执行搜索、计算、数据库访问等具体任务；
- **记忆（Memory）**：存储上下文对话信息，支持多轮对话；
- **决策逻辑**：决定当前应调用哪个工具或如何响应。
''')

## 🧪 Step 2：环境配置（必做）

运行以下代码，安装必要库并配置OpenAI的API Key：

In [ ]:
%pip install langchain openai faiss-cpu tiktoken langchain-community --quiet

In [ ]:
from langchain.chat_models import ChatOpenAI

# 请根据你的OpenAI账号填写以下信息
OPENAI_API_KEY = "2c2a258f6daf4ab1a7eaa1f1298f5a0d.Lajx17gMNJr4bk9F"
OPENAI_API_BASE = "https://open.bigmodel.cn/api/paas/v4/"  # 如果你使用的是代理或自定义服务商，可以修改此URL
OPENAI_MODEL = "glm-4-air" 

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_API_BASE"] = OPENAI_API_BASE

llm = ChatOpenAI(
    model_name=OPENAI_MODEL,
    temperature=0.9,
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE
)

## 🧱 Step 3：构建你的第一个智能体

### 🛠️ 第一步：定义工具（Tool）

我们将实现一个加法器工具。

In [ ]:
from langchain.tools import tool

@tool
def add_numbers(input: str) -> str:
    """接收两个用逗号分隔的数字字符串，如 '3,4' 返回 '7'。支持容错与格式清洗。"""
    try:
        # 移除引号、换行、冒号等常见格式污染
        input = input.strip().replace("'", "").replace("\"", "").replace("\n", "").replace("Observation:", "")
        parts = [x.strip() for x in input.split(',') if x.strip()]
        if len(parts) != 2:
            return "输入格式错误：请提供两个用逗号分隔的数字，如 '3,4'"
        a, b = parts
        return str(int(a) + int(b))
    except Exception as e:
        return f"输入格式错误，请输入形如 '3,4' 的字符串：{str(e)}"

### 🤖 第二步：初始化Agent

In [ ]:
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType

# 添加工具列表
tools = [
    Tool(
        name="AddNumbers",
        func=add_numbers,
        description="用于执行两个数字加法，例如输入 '3,4'"
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

### 🧪 第三步：运行测试
langchain中的agent遵循ReAct范式

相关阅读：ReAct: Synergizing Reasoning and Acting in Language Models

In [ ]:
agent.run("请帮我计算3加7")

📝 **问题思考**：你能修改输入内容让Agent报错吗？为什么会出错？

🧠 **自主练习**：请你实现一个阶乘计算工具 `factorial_tool`，接收一个正整数字符串返回其阶乘值，如输入 '5' 应返回 '120'。要求：
- 检查输入是否为非负整数
- 对非法输入添加适当错误提示
- 将其添加至 `tools` 列表

## 🔁 Step 4：丰富你的Agent记忆能力

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

agent_with_memory = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

In [ ]:
agent_with_memory.run("请你帮我计算12+47等于多少")

In [ ]:
agent_with_memory.run("我刚才的计算结果是多少？")

In [ ]:
agent_with_memory.run("在刚才的基础上再加23，结果是多少？")

In [ ]:
print("\n--- 当前记忆内容 ---")
for msg in memory.chat_memory.messages:
    print(f"{msg.type.upper()}: {msg.content}")

### 使用摘要记忆
摘要记忆通过将对话历史生成摘要，适合长对话中保持语义一致性而不被信息爆炸干扰。

In [ ]:
from langchain.memory import ConversationSummaryMemory

summary_memory = ConversationSummaryMemory(llm=llm, memory_key="summary_chat_history")

agent_summary = initialize_agent(
    tools=tools,
    llm=llm,
    memory=summary_memory,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)

**练习**：试试连续输入三轮关于旅游计划的问题，观察模型是否能总结用户的目标。

## 🎯 自主练习
请完成以下任务，提交代码与说明：

1. **集成搜索功能**，使用 `DuckDuckGoSearchRun` 工具并问“OpenAI是谁？”；
2. **尝试更换Agent类型**为 `AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION` 并观察变化；

提示：使用如下代码添加搜索工具：
```python
!pip install -U duckduckgo-search
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
tools.append(Tool(name="Search", func=search.run, description="执行网页搜索"))
```


## 📌思考
- 如果要让智能体拥有“规划一周学习任务”的能力，你会添加哪些工具和模块？